# SUPG stabilization of the advection-diffusion-reaction equation

strong form

$$
\begin{align*}
&\text{Find $u(\textbf{x}): \Omega \to \mathbb{R}$ such that } \\
&\begin{cases}
\frac{\partial u}{\partial t} + \textbf{a}\cdot\nabla u= \nabla\cdot(\mathsf{D}\cdot\nabla u) + Ru + J & \forall\textbf{x}\in\Omega \\
u=u_{\text{D}} & \forall \textbf{x}\in\partial\Omega_{\text{D}} \\
\textbf{n}\cdot(\mathsf{D}\cdot\nabla{u}) = u_{\text{N}} & \forall\textbf{x}\in\partial\Omega_{\text{N}}=\partial\Omega/\partial\Omega_{\text{D}}
\end{cases}~.
\end{align*}
$$

## Advection-diffusion of a Gaussian profile on an interval

Donea, J. & Huerta, A. (2003). *Finite Element Methods for Flow Problems*. $\S 5.6.1$

$$
\mathbb{S}
\begin{cases}
\Omega = [0, 1] \\
u_0(x) = U_0\exp\left(-(x-x_0)^2/\sigma^2\right) \\
u_{\text{D}}(x=0,1)=0 \\
\textbf{a}=a\,\textbf{e}_x \\
\mathsf{D}=D\mathsf{I} \\
R=0 \\
J = 1 \\
u_{\text{e}}(x, t)=\frac{U_0}{\sqrt{1+4Dt/\sigma^2}}\exp\left(-\frac{(x-x_0-at)^2}{\sigma^2+4Dt}\right)
\end{cases}
$$

In [ ]:
import numpy as np

from lucifex.mesh import interval_mesh
from lucifex.fem import Constant
from lucifex.fdm import (
    CN, BE, FE, cfl_timestep,
    FiniteDifference, FunctionSeries, ConstantSeries, finite_difference_order,
)
from lucifex.solver import ibvp, evaluation , BoundaryConditions
from lucifex.sim import Simulation, run
from lucifex.viz import plot_line, create_multifigure
from lucifex.io import write, get_ipynb_file_name
from lucifex.utils import nested_dict
from lucifex.pde.advection_diffusion import advection_diffusion
from lucifex.pde.supg import peclet


def create_simulation(
    supg: str | None,
    Nx: int,
    a: float,
    d: float,
    courant: float,
    D_adv: FiniteDifference,
    D_diff: FiniteDifference,
    Lx: float = 1.0,
    U0: float = 5/7,
    x0: float = 0.1,
    sigma: float = 0.03,
    cache_matrix: bool = False,
) -> Simulation:
    order = finite_difference_order(D_adv, D_diff)
    mesh = interval_mesh(Lx, Nx)

    t = ConstantSeries(mesh, name='t', ics=0.0)
    dt = Constant(mesh, cfl_timestep(a, Lx/Nx, courant), name='dt')

    a = Constant(mesh, (a, 0), name='a')
    u = FunctionSeries((mesh, 'P', 1), name='u', order=order, store=1)

    ics = lambda x: U0 * np.exp(-(x[0] - x0)**2 / sigma **2)
    bcs = BoundaryConditions(
        ('dirichlet', lambda x: x[0], 0.0),
        ('dirichlet', lambda x: x[0] - Lx, 0.0),
    )
    u_solver = ibvp(advection_diffusion, ics, bcs, cache_matrix=cache_matrix)(
        u, dt, a, d, D_adv, D_diff, supg=supg,
    )

    return Simulation([u_solver], t, dt)


def exact_gaussian(
):
    ...


Lx = 1.0
Nx = 50
h = Lx / Nx
a = 1.0
courant = 1.0

d_opts = (0.2, 1/18, 1/100)
D_adv_diff_opts = ((CN, CN), (FE, FE), (BE, BE), (BE, CN))
supg_opts = [None, 'coth']
simulations = nested_dict((float, tuple, str, Simulation))

for supg in supg_opts:
    for d in d_opts:
        for (D_adv, D_diff) in D_adv_diff_opts:
            simulations[supg][d][(D_adv, D_diff)] = create_simulation(supg, Nx, a, d, courant, D_adv, D_diff)


t_stop = 0.6
for supg in supg_opts:
    for d in d_opts:
        for (D_adv, D_diff) in D_adv_diff_opts:
            run(simulations[supg][d][(D_adv, D_diff)], t_stop=t_stop) 
